# Sentiment Analysis: Thai Text Classification

## Objective
สร้างระบบวิเคราะห์ความรู้สึก (Sentiment Analysis) เพื่อจำแนกรีวิวเป็น **positive**, **neutral**, หรือ **negative**

## Approach
ใช้ **Pre-trained Model** จาก Hugging Face ที่ถูก fine-tuned สำหรับภาษาไทยแล้ว:
- **Model**: WangchanBERTa (fine-tuned for sentiment analysis)
- **Dataset**: Wisesight Sentiment Corpus (Thai social media text)

## Evaluation Metrics
- Accuracy
- Precision
- Recall
- F1-score
- Confusion Matrix

## 1. Install and Import Libraries

In [3]:
# Install required packages (uncomment if needed)
# !pip install transformers datasets torch scikit-learn matplotlib seaborn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Hugging Face libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Scikit-learn for evaluation
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

print("Libraries imported successfully!")

ModuleNotFoundError: No module named 'matplotlib'

## 2. Load Dataset

ใช้ **Wisesight Sentiment Corpus** - ข้อความภาษาไทยจาก Social Media พร้อม labels:
- `pos` (positive)
- `neu` (neutral)
- `neg` (negative)
- `q` (question) - จะถูกกรองออก

In [ ]:
# Load Wisesight Sentiment dataset from Hugging Face
print("Loading Wisesight Sentiment dataset...")
dataset = load_dataset("wisesight_sentiment")

print(f"\nDataset structure:")
print(dataset)

print(f"\nTrain set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

In [ ]:
# Explore the dataset
print("Sample data from training set:")
print("=" * 60)

# Label mapping
label_names = {0: 'neg', 1: 'neu', 2: 'pos', 3: 'q'}

for i in range(5):
    sample = dataset['train'][i]
    print(f"\nText: {sample['texts'][:100]}...")
    print(f"Label: {label_names[sample['category']]}")
    print("-" * 40)

In [ ]:
# Check label distribution
train_labels = [label_names[x['category']] for x in dataset['train']]
test_labels = [label_names[x['category']] for x in dataset['test']]

print("Label distribution in training set:")
print(pd.Series(train_labels).value_counts())

print("\nLabel distribution in test set:")
print(pd.Series(test_labels).value_counts())

## 3. Data Preprocessing

กรองเอาเฉพาะ 3 classes: positive, neutral, negative (ไม่รวม question)

In [ ]:
def filter_and_prepare_data(dataset_split):
    """
    Filter out 'question' category and prepare data for evaluation.
    
    Args:
        dataset_split: HuggingFace dataset split
    
    Returns:
        texts: List of text strings
        labels: List of label strings (neg, neu, pos)
    """
    texts = []
    labels = []
    
    label_map = {0: 'negative', 1: 'neutral', 2: 'positive', 3: 'question'}
    
    for item in dataset_split:
        label = label_map[item['category']]
        # Skip question category
        if label != 'question':
            texts.append(item['texts'])
            labels.append(label)
    
    return texts, labels


# Prepare test data (filter out questions)
test_texts, test_labels = filter_and_prepare_data(dataset['test'])

print(f"Test samples after filtering: {len(test_texts)}")
print(f"\nLabel distribution after filtering:")
print(pd.Series(test_labels).value_counts())

## 4. Load Pre-trained Model

ใช้ **WangchanBERTa** ที่ถูก fine-tuned สำหรับ sentiment analysis บน Wisesight dataset

Model นี้มี accuracy ~91% บน Wisesight dataset

In [ ]:
# Load pre-trained sentiment model
MODEL_NAME = "poom-sci/WangchanBERTa-finetuned-sentiment"

print(f"Loading model: {MODEL_NAME}")
print("This may take a minute...")

# Create sentiment analysis pipeline
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    max_length=512,
    truncation=True
)

print("Model loaded successfully!")

In [ ]:
# Test the model with sample texts
sample_texts = [
    "อาหารอร่อยมาก บริการดีเยี่ยม แนะนำเลยครับ",
    "ร้านนี้ธรรมดามาก ไม่มีอะไรพิเศษ",
    "แย่มาก รอนานมาก อาหารไม่อร่อย ไม่มาอีกแล้ว",
    "ของดีราคาถูก คุ้มค่ามากๆ",
    "ผิดหวังมาก ไม่เหมือนในรูปเลย"
]

print("Testing model with sample texts:")
print("=" * 60)

for text in sample_texts:
    result = sentiment_analyzer(text)[0]
    print(f"\nText: {text}")
    print(f"Prediction: {result['label']} (confidence: {result['score']:.4f})")

## 5. Run Predictions on Test Set

ทำนายผลบน test set เพื่อประเมินประสิทธิภาพของ model

In [ ]:
def predict_sentiment_batch(texts: List[str], batch_size: int = 32) -> List[str]:
    """
    Predict sentiment for a list of texts in batches.
    
    Args:
        texts: List of text strings
        batch_size: Number of texts to process at once
    
    Returns:
        List of predicted labels
    """
    predictions = []
    total = len(texts)
    
    for i in range(0, total, batch_size):
        batch = texts[i:i+batch_size]
        results = sentiment_analyzer(batch)
        
        for result in results:
            # Map model output to standard labels
            label = result['label'].lower()
            if 'pos' in label:
                predictions.append('positive')
            elif 'neg' in label:
                predictions.append('negative')
            else:
                predictions.append('neutral')
        
        # Progress update
        if (i + batch_size) % 100 == 0 or (i + batch_size) >= total:
            print(f"Progress: {min(i + batch_size, total)}/{total} samples processed")
    
    return predictions


# Run predictions on test set
print("Running predictions on test set...")
print("=" * 60)

predicted_labels = predict_sentiment_batch(test_texts)

print(f"\nPrediction complete! Total predictions: {len(predicted_labels)}")

## 6. Evaluation Metrics

คำนวณ metrics ต่างๆ เพื่อประเมินประสิทธิภาพ:
- **Accuracy**: สัดส่วนที่ทำนายถูก
- **Precision**: ความแม่นยำของการทำนาย positive
- **Recall**: ความครบถ้วนในการจับ positive จริง
- **F1-score**: Harmonic mean ของ precision และ recall

In [ ]:
# Calculate metrics
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels, average='weighted')
recall = recall_score(test_labels, predicted_labels, average='weighted')
f1 = f1_score(test_labels, predicted_labels, average='weighted')

print("=" * 60)
print("EVALUATION METRICS")
print("=" * 60)
print(f"\nAccuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")

In [ ]:
# Detailed classification report
print("\n" + "=" * 60)
print("CLASSIFICATION REPORT")
print("=" * 60)
print(classification_report(test_labels, predicted_labels, digits=4))

## 7. Confusion Matrix

แสดง Confusion Matrix เพื่อดูรายละเอียดการทำนาย

In [ ]:
# Create confusion matrix
labels_order = ['negative', 'neutral', 'positive']
cm = confusion_matrix(test_labels, predicted_labels, labels=labels_order)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=labels_order,
    yticklabels=labels_order
)
plt.title('Confusion Matrix - Sentiment Analysis', fontsize=14)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

print("Confusion matrix saved to: confusion_matrix.png")

In [ ]:
# Normalized confusion matrix (percentage)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm_normalized, 
    annot=True, 
    fmt='.2%', 
    cmap='Greens',
    xticklabels=labels_order,
    yticklabels=labels_order
)
plt.title('Normalized Confusion Matrix (Percentage)', fontsize=14)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.savefig('confusion_matrix_normalized.png', dpi=150)
plt.show()

## 8. Error Analysis

วิเคราะห์ตัวอย่างที่ model ทำนายผิด

In [ ]:
# Find misclassified examples
errors = []
for i, (text, true_label, pred_label) in enumerate(zip(test_texts, test_labels, predicted_labels)):
    if true_label != pred_label:
        errors.append({
            'text': text[:100] + '...' if len(text) > 100 else text,
            'true_label': true_label,
            'predicted_label': pred_label
        })

print(f"Total misclassified samples: {len(errors)} / {len(test_texts)} ({len(errors)/len(test_texts)*100:.2f}%)")

# Show some error examples
print("\n" + "=" * 60)
print("SAMPLE MISCLASSIFICATIONS")
print("=" * 60)

for i, error in enumerate(errors[:10]):
    print(f"\n[{i+1}] Text: {error['text']}")
    print(f"    True: {error['true_label']} | Predicted: {error['predicted_label']}")

## 9. Interactive Demo

ทดสอบ model กับข้อความใหม่

In [ ]:
def analyze_sentiment(text: str) -> Dict:
    """
    Analyze sentiment of a single text.
    
    Args:
        text: Input text string
    
    Returns:
        Dictionary with sentiment and confidence
    """
    result = sentiment_analyzer(text)[0]
    
    # Map to standard labels
    label = result['label'].lower()
    if 'pos' in label:
        sentiment = 'positive'
    elif 'neg' in label:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    
    return {
        'text': text,
        'sentiment': sentiment,
        'confidence': result['score']
    }


# Test with custom texts
custom_texts = [
    "สินค้าคุณภาพดีมาก ส่งเร็วมากๆ ประทับใจค่ะ",
    "ปกติธรรมดา ไม่ได้รู้สึกว่าดีหรือแย่",
    "ห่วยแตกมาก ไม่คุ้มราคาเลย อย่าซื้อ",
    "พนักงานน่ารักมาก ช่วยเหลือดี",
    "รสชาติพอใช้ได้ แต่ราคาแพงไป"
]

print("=" * 60)
print("SENTIMENT ANALYSIS DEMO")
print("=" * 60)

for text in custom_texts:
    result = analyze_sentiment(text)
    emoji = {'positive': '😊', 'neutral': '😐', 'negative': '😞'}[result['sentiment']]
    print(f"\n{emoji} [{result['sentiment'].upper()}] (conf: {result['confidence']:.2%})")
    print(f"   \"{text}\"")

## 10. Summary

สรุปผลการทดสอบ Sentiment Analysis

In [ ]:
print("=" * 60)
print("SENTIMENT ANALYSIS SUMMARY")
print("=" * 60)

print(f"\n📊 Dataset: Wisesight Sentiment Corpus")
print(f"🤖 Model: {MODEL_NAME}")
print(f"📝 Test samples: {len(test_texts)}")

print(f"\n📈 Performance Metrics:")
print(f"   • Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   • Precision: {precision:.4f}")
print(f"   • Recall:    {recall:.4f}")
print(f"   • F1-score:  {f1:.4f}")

print(f"\n📁 Output files:")
print(f"   • confusion_matrix.png")
print(f"   • confusion_matrix_normalized.png")

print("\n" + "=" * 60)

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame({
    'text': test_texts,
    'true_label': test_labels,
    'predicted_label': predicted_labels
})

# Save to CSV
results_df.to_csv('sentiment_predictions.csv', index=False, encoding='utf-8-sig')
print("Predictions saved to: sentiment_predictions.csv")

# Display sample
results_df.head(10)